<a href="https://colab.research.google.com/github/luispabloaiello-da/data-cleaning-pandas/blob/main/Shark_Mini_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np
import re

# Specify the full path to your Excel file
url = "https://www.sharkattackfile.net/spreadsheets/GSAF5.xls"
pd.read_excel(url)

# Read the Excel file into a DataFrame
df = pd.read_excel(url)

#
display(df.info())
display(df.dtypes)
display(df.nunique())
display(df.describe())
display(df.describe(include='all'))
display((df.isna().sum()))
display((df.isna().sum()/df.shape[0])*100)
display(df)

# Clean up column names: remove spaces, convert to lowercase, replace spaces with underscores
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

# Drop empty/redundant columns
# Check which columns have almost all missing values
print()
print('# Drop empty/redundant columns')
print('# Check which columns have almost all missing values')
display(df.isnull().sum(), (df.isna().sum()/df.shape[0])*100)
print()

# Drop columns that are almost entirely NaN or are meta-data
drop_cols = ['unnamed:_21', 'unnamed:_22', 'pdf', 'href_formula', 'href', 'case_number.1', 'original_order']
df = df.drop(columns=drop_cols, errors='ignore')

# Drops columns with more than `threshold` percent missing values.
# Returns a cleaned DataFrame.
def drop_columns_missing_pct(df, threshold=30):
    missing_pct = (df.isnull().sum() / df.shape[0]) * 100
    cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()
    return df.drop(columns=cols_to_drop)

df = drop_columns_missing_pct(df, threshold=30)

# # Convert 'age' to integer, set invalid entries ('?', 'Not stated') as NaN
# df['age'] = pd.to_numeric(df['age'], errors='coerce').astype('Int64')

# Impute 'Age' with median, categorical with 'Unknown'
if 'age' in df.columns:
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    df['age'] = df['age'].fillna(df['age'].median())

# Standardize categorical columns: Sex and Country
df['sex'] = df['sex'].str.strip().str.upper().replace({'male': 'M', 'female': 'F', 'nan': None})
df['country'] = df['country'].str.strip().str.title() # Makes "usa" -> "Usa", "AUSTRALIA" -> "Australia"

# For column 'type', unify spelling and capitalization
df['type'] = df['type'].str.strip().str.capitalize()

# Convert 'year' to integer, handle missing/invalid as NaN
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')

# Convert 'date' to datetime (custom parsing required for inconsistent formats)
# Try to parse common date formats, fallback to NaT
# df['date_parsed'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)

# drop_duplicates() with no arguments checks all columns.
# Only rows where every column value matches another row are considered duplicates.
# It keeps the first occurrence and removes subsequent identical row
df = df.drop_duplicates()

cat_cols = list(df.select_dtypes(include=['object']))
# cat_cols = ['type', 'country', 'state', 'location', 'activity', 'name', 'sex', 'injury', 'fatal_y/n', 'species']
for col in cat_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

# Extract main activity using regex, e.g., 'Surfing', 'Swimming', 'Fishing'
df['activity_cat'] = df['activity'].str.lower().str.extract('(surf|swim|fish|dive|wade|kayak|boating)', expand=False)
df['activity_cat'] = df['activity_cat'].fillna('other')

display(df['fatal_y/n'].nunique())
display(df['fatal_y/n'].unique())
display((df['fatal_y/n'].value_counts()))

# Standarize values in column fatal_y/n using regex
# Converts input to uppercase string and strips whitespace.
# Returns 'Y' for any value with 'Y', 'F', or starting with 'Y' (e.g. 'Y x 2')
# Returns 'N' for 'N', 'M', or starting with 'N'
# Returns 'Unknown' for ambiguous, unknown, missing, or other values.

def clean_fatal_yn(val):
    if pd.isnull(val):
        return 'Unknown'
    val_str = str(val).strip().upper()
    if re.search(r'\bY\b', val_str) or re.search(r'\bF\b', val_str) or re.match(r'Y', val_str):
        return 'Y'
    if re.search(r'\bN\b', val_str) or re.search(r'\bM\b', val_str) or re.match(r'N', val_str):
        return 'N'
    if val_str in ['UNKNOWN', '', 'NQ']:
        return 'Unknown'
    return 'Unknown'

df['clean_fatal_yn'] = df['fatal_y/n'].apply(clean_fatal_yn)

display(df[['fatal_y/n', 'clean_fatal_yn']].value_counts())
display(df[['fatal_y/n', 'clean_fatal_yn']].nunique())

# This function takes a row from a pandas DataFrame containing shark attack data, and tries to create a clean,
# standardized date (date_parsed) for each entry. It does this by:

# Extracting a year from the date string or using the year column, if available and valid.
# Attempting to parse the full date string using several common date formats.
# If only the year is known, it defaults the date to January 1st of that year.
# If no valid date or year is found, it returns a missing value (NaT).
# This ensures that all entries have a consistent datetime value for analysis,
# even when the original data contains incomplete or messy date information.

def parse_date(row):
    date_str = str(row['date']).strip()
    year_val = row['year']

    # Helper: is the year valid?
    def valid_year(y):
        return pd.notnull(y) and isinstance(y, (int, float)) and 1000 <= int(y) <= 2025

    # 1. Try to extract a 4-digit year from the date string
    year_from_date = None
    match_year = re.search(r'(19|20)\d{2}', date_str)
    if match_year:
        year_from_date = int(match_year.group(0))
    # Try to match a year range like '1900-1905' and extract the first year
    match_range = re.match(r'(\d{4})-(\d{4})', date_str)
    if match_range:
        year_from_date = int(match_range.group(1))

    # 2. Check for day/month pattern but NO year (like '17th August')
    match_day_month = re.match(r'(\d{1,2})(?:st|nd|rd|th)?\s+([A-Za-z]+)', date_str)
    if match_day_month and not year_from_date:
        day = int(match_day_month.group(1))
        month = match_day_month.group(2)
        # Use year from year column if valid
        if valid_year(year_val):
            try:
                # Build date string: '17 August 2025'
                full_date_str = f"{day} {month} {int(year_val)}"
                parsed = pd.to_datetime(full_date_str, format="%d %B %Y", errors='coerce')
                if pd.notnull(parsed):
                    return parsed.strftime('%d-%m-%Y')
            except Exception:
                pass

    # 3. Try to parse date string directly (may include year)
    parsed = pd.to_datetime(date_str, errors='coerce', dayfirst=None)
    if pd.isnull(parsed):
        # Try several common explicit formats
        for fmt in ("%Y-%m-%d %H:%M:%S", "%Y.%m.%d", "%m-%d-%Y", "%d %B %Y", "%d %b %Y"):
            try:
                parsed = pd.to_datetime(date_str, format=fmt, errors='coerce')
                if pd.notnull(parsed):
                    break
            except Exception:
                continue

    # 4. Determine a valid year
    if year_from_date and valid_year(year_from_date):
        valid_year_val = int(year_from_date)
    elif valid_year(year_val):
        valid_year_val = int(year_val)
    else:
        valid_year_val = None

    # 5. Return final output in "YYYY-MM-DD"
    if pd.notnull(parsed):
        return parsed.strftime('%d-%m-%Y')
    elif valid_year_val:
        return f"01-01-{valid_year_val}"  # Only year known, set to Jan 1
    else:
        return 'Unknown'  # If nothing is valid, return str 'Unknown'

# Usage:
df['date_parsed'] = df.apply(parse_date, axis=1)

display(df[['date', 'year', 'date_parsed']])
display(df[['date', 'year', 'date_parsed']].nunique())

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7042 entries, 0 to 7041
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            7042 non-null   object 
 1   Year            7040 non-null   float64
 2   Type            7024 non-null   object 
 3   Country         6992 non-null   object 
 4   State           6557 non-null   object 
 5   Location        6475 non-null   object 
 6   Activity        6457 non-null   object 
 7   Name            6823 non-null   object 
 8   Sex             6463 non-null   object 
 9   Age             4048 non-null   object 
 10  Injury          7007 non-null   object 
 11  Fatal Y/N       6481 non-null   object 
 12  Time            3516 non-null   object 
 13  Species         3911 non-null   object 
 14  Source          7022 non-null   object 
 15  pdf             6799 non-null   object 
 16  href formula    6794 non-null   object 
 17  href            6796 non-null   o

None

,0
Date,object
Year,float64
Type,object
Country,object
State,object
Location,object
Activity,object
Name,object
Sex,object
Age,object


,0
Date,6086
Year,261
Type,13
Country,247
State,937
Location,4593
Activity,1606
Name,5767
Sex,10
Age,250


,Year,original order
count,7040.000000,6799.000000
mean,1935.621449,3401.152081
std,271.221061,1963.076319
min,0.000000,2.000000
25%,1948.000000,1701.500000
50%,1986.000000,3401.000000
75%,2010.000000,5100.500000
max,2026.000000,6802.000000


...

,,count
fatal_y/n,clean_fatal_yn,
N,N,4908
Y,Y,1481
Unknown,Unknown,561
UNKNOWN,Unknown,71
N,N,7
F,Y,5
M,N,3
2017,Unknown,1
Nq,N,1


,0
fatal_y/n,13
clean_fatal_yn,3


,date,year,date_parsed
0,16th August 2025,2025,16-08-2025
1,18th August,2025,18-08-2025
2,17th August,2025,17-08-2025
3,7th August,2025,07-08-2025
4,1st August,2025,01-08-2025
...,...,...,...
7037,Before 1903,0,01-01-1903
7038,Before 1903,0,01-01-1903
7039,1900-1905,0,01-01-1900
7040,1883-1889,0,01-01-1883


,0
date,6086
year,261
date_parsed,5276


,date,year,type,country,state,location,activity,name,sex,injury,fatal_y/n,source,case_number,activity_cat,clean_fatal_yn,date_parsed
0,16th August 2025,2025,Provoked,Usa,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,Laceration to right leg below the knee,N,Johannes Marchand: Kevin McMurray Trackingshar...,Unknown,fish,N,16-08-2025
1,18th August,2025,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,None sustained board severly damaged,N,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,Unknown,surf,N,18-08-2025
2,17th August,2025,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,Severe injuries no detail,N,Ralph Collier GSAF and Kevin MCMurray Tracking...,Unknown,fish,N,17-08-2025
3,7th August,2025,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,None sustained board severely damaged,N,Bob Myatt GSAF,Unknown,surf,N,07-08-2025
4,1st August,2025,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,Bite to thigh area,N,Kevin McMurray Trackingsharks.com: NY Post,Unknown,other,N,01-08-2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,Before 1903,0,Unprovoked,Australia,Western Australia,Roebuck Bay,Diving,male,M,FATAL,Y,"H. Taunton; N. Bartlett, p. 234",ND.0005,other,Y,01-01-1903
7038,Before 1903,0,Unprovoked,Australia,Western Australia,Unknown,Pearl diving,Ahmun,M,FATAL,Y,"H. Taunton; N. Bartlett, pp. 233-234",ND.0004,other,Y,01-01-1903
7039,1900-1905,0,Unprovoked,Usa,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,FATAL,Y,"F. Schwartz, p.23; C. Creswell, GSAF",ND.0003,swim,Y,01-01-1900
7040,1883-1889,0,Unprovoked,Panama,Unknown,"Panama Bay 8ºN, 79ºW",Unknown,Jules Patterson,M,FATAL,Y,"The Sun, 10/20/1938",ND.0002,other,Y,01-01-1883


In [ ]:
1 import pandas etc...
2 load dataset & 3 explore the data (also find missing values)
3 explore the data (also find missing values)
4 data cleaning (non date fields, nulls duplicates strings)
5 drop the duplicates ( fill missing categorical and values and string clean )

# Shark Attacks - Data Wrangling Quest

## Overview
This project cleans and analyzes the Shark Attack dataset to uncover patterns and insights, supporting a business hypothesis.

## Structure
- **Mini_Project_1_Shark_Attacks.ipynb**: Main workflow notebook
- **cleaning.py**: Modular cleaning functions
- **eda.py**: Basic EDA and visualization functions
- **shark_attacks_clean.csv**: Cleaned data (exported)
- **slides/**: Link to final presentation slides

## Instructions
1. Open the notebook and run each cell step-by-step.
2. Cleaning functions are defined in `cleaning.py`, called via `main_cleaning_function(df)`.
3. EDA visuals are generated with `basic_eda(df_clean)` from `eda.py`.
4. Update hypotheses and business case in the notebook and presentation.

## Deliverables
- Jupyter notebook (with code & results)
- Modular `.py` files
- Cleaned CSV
- README & documentation
- Online slide deck (link here: [Your Google Slides URL])

## Authors
- [Your Names Here]
